In [3]:
#Installing the dask package
!pip install dask

In [4]:
#Pulling required packages--
import pandas as pd
import math
import numpy as np 
import warnings
import dask.dataframe as dd
import dask.array as da
import dask.bag as db

warnings.filterwarnings("ignore")
pd.set_option('display.max_rows', None)

In [5]:
#Pulling Data from Issues Table-
issues_df = pd.read_csv('github_issues.csv')  
print("Pulling The Issues df:",issues_df.shape)


#Pulling Data from Commits Table-
commits_df = pd.read_csv('github_commits.csv')  
print("Pulling the Commits df:",commits_df.shape)


#Pulling Data from Main Table-
main_df = pd.read_csv('github_main.csv')  

print("Pulling the Main df:",main_df.shape)


Pulling The Issues df: (118, 13)
Pulling the Commits df: (624, 12)
Pulling the Main df: (23, 16)


In [6]:
ddf = dd.from_pandas(issues_df, npartitions=10)
ddf

,id,number,title,created_at,updated_at,closed_at,author_association,comments,body,user_login,user_id,Crypto_Name,Extension
npartitions=10,,,,,,,,,,,,,
0,int64,int64,object,object,object,float64,object,int64,object,object,int64,object,object
12,...,...,...,...,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
108,...,...,...,...,...,...,...,...,...,...,...,...,...
117,...,...,...,...,...,...,...,...,...,...,...,...,...


In [6]:
#Sample df display----
#data=main_df[main_df['Crypto']=='bitcoin']
#data=commits_df[commits_df['Crypto_Name']=='solana']
data=issues_df[issues_df['Crypto_Name']=='solana']

data.head(2)

,id,number,title,created_at,updated_at,closed_at,author_association,comments,body,user_login,user_id,Crypto_Name,Extension
0,1661541619,31138,v1.14: doc: typo in geyser plugin doc (backpor...,2023-04-10 23:29:19,2023-04-10 23:30:21,NaN,CONTRIBUTOR,0,This is an automatic backport of pull request ...,mergify[bot],37929162,solana,/issues
1,1661342491,31137,validator: `--tpu-host-addr` -> `--public-tpu-...,2023-04-10 20:49:31,2023-04-11 00:04:07,NaN,CONTRIBUTOR,1,#### Problem\r\n\r\n`--tpu-host-addr` is not v...,diman-io,71597545,solana,/issues


In [7]:
data=ddf

In [20]:
ddf.id.isnull().sum().compute()

0

In [22]:
#Code for Univariate analysis---

#### code for catagorical data
data_file = data.select_dtypes(exclude =np.number)
def count_stats(column_data):
    total_count = column_data.count().compute()
    unique_count = column_data.nunique().compute()
    null_count = column_data.isnull().sum().compute()

    ## printing column counts
    #print('Total number of values : {}'.format(total_count))
    #print('Total number of unique values : {}'.format(unique_count))
    #print('Total number of null values : {}'.format(null_count))

    return total_count, unique_count, null_count

discrete_dataFrame_columns = ['Column Name', 'total_count', 'unique_count', 'null_count', '% null', 'levels',
                                  'value_count', '% counts']
discrete_dataFrame = pd.DataFrame(columns=discrete_dataFrame_columns)

def count_stats(column_data):
    total_count = column_data.count().compute()
    unique_count = column_data.nunique().compute()
    null_count = column_data.isnull().sum().compute()

    ## printing column counts
    #print('Total number of values : {}'.format(total_count))
    #print('Total number of unique values : {}'.format(unique_count))
    #print('Total number of null values : {}'.format(null_count))

    return total_count, unique_count, null_count
for column in data_file.columns:

        #print("Working on : " + column)

        column_data = data_file[column]
        total_count, unique_count , null_count = count_stats(column_data)
        if(total_count!=0):
            null_per = float(null_count)/float(total_count)*100
        else:
            null_per = 0

        #column_type,column_data = get_column_type(column_data)


        value_counts = pd.DataFrame(column_data.value_counts())
        value_counts = value_counts.reset_index()
        value_counts.insert(loc=0,column='Column Name',value=column)
        value_count_columns = ['Column Name','levels', 'value_count']
        value_counts.columns = value_count_columns
        value_counts = value_counts.sort_values(by='value_count', ascending=False)
        value_counts_top = value_counts[:10].copy()
        top_frequency = value_counts_top.value_count.sum()
        others_frequency = value_counts[10:].value_count.sum()


        temp_column_names = ['Column Name','total_count','unique_count','null_count','% null']
        temp_values = [[column,total_count,unique_count,null_count,null_per]]
        temp_dataFrame = pd.DataFrame(columns=temp_column_names,data=temp_values)

        ## create value counts dataframe with others column
        value_counts_top = value_counts_top.append(pd.DataFrame(columns=value_count_columns,data=[[column,'others',others_frequency]]))
        ## add null level
        value_counts_top = value_counts_top.append(pd.DataFrame(columns=value_count_columns,data=[[column,'nulls',null_count]]))
        ## add % counts

        if(total_count!=0):
                value_counts_top['% counts'] = value_counts_top.value_count/float(total_count) * 100
        else:
                value_counts_top['% counts'] = 0

            ## merge temp_dataframe and value_counts_top, then insert into discrete_dataframe
        temp_discrete_dataFrame = temp_dataFrame.merge(how='left',left_on='Column Name',right_on='Column Name',right=value_counts_top)

        discrete_dataFrame = discrete_dataFrame.append(temp_discrete_dataFrame)
discrete_dataFrame_cat = discrete_dataFrame.copy()
discrete_dataFrame_cat['Column_Type']=['Categorical']*len(discrete_dataFrame_cat)        
discrete_dataFrame_cat = discrete_dataFrame_cat[['Column_Type','Column Name', 'total_count', 'unique_count', 'null_count', '% null',
       'levels', 'value_count', '% counts']]
        
#### code for cont. data
data_file = data.select_dtypes(include =np.number)
def count_stats(column_data):
    total_count = column_data.count().compute()
    unique_count = column_data.nunique().compute()
    null_count = column_data.isnull().sum().compute()

    ## printing column counts
    #print('Total number of values : {}'.format(total_count))
    #print('Total number of unique values : {}'.format(unique_count))
    #print('Total number of null values : {}'.format(null_count))

    return total_count, unique_count, null_count

discrete_dataFrame_columns = ['Column Name', 'total_count', 'unique_count', 'null_count', '% null', 'levels',
                                  'value_count', '% counts']
discrete_dataFrame = pd.DataFrame(columns=discrete_dataFrame_columns)

def count_stats(column_data):
    total_count = column_data.count().compute()
    unique_count = column_data.nunique().compute()
    null_count = column_data.isnull().sum().compute()

    ## printing column counts
    #print('Total number of values : {}'.format(total_count))
    #print('Total number of unique values : {}'.format(unique_count))
    #print('Total number of null values : {}'.format(null_count))

    return total_count, unique_count, null_count
for column in data_file.columns:

       #print("Working on : " + column)

        column_data = data_file[column]
        total_count, unique_count , null_count = count_stats(column_data)
        if(total_count!=0):
            null_per = float(null_count)/float(total_count)*100
        else:
            null_per = 0

        #column_type,column_data = get_column_type(column_data)


        value_counts = pd.DataFrame(column_data.value_counts())
        value_counts = value_counts.reset_index()
        value_counts.insert(loc=0,column='Column Name',value=column)
        value_count_columns = ['Column Name','levels', 'value_count']
        value_counts.columns = value_count_columns
        value_counts = value_counts.sort_values(by='value_count', ascending=False)
        value_counts_top = value_counts[:10].copy()
        top_frequency = value_counts_top.value_count.sum()
        others_frequency = value_counts[10:].value_count.sum()


        temp_column_names = ['Column Name','total_count','unique_count','null_count','% null']
        temp_values = [[column,total_count,unique_count,null_count,null_per]]
        temp_dataFrame = pd.DataFrame(columns=temp_column_names,data=temp_values)

        ## create value counts dataframe with others column
        value_counts_top = value_counts_top.append(pd.DataFrame(columns=value_count_columns,data=[[column,'others',others_frequency]]))
        ## add null level
        value_counts_top = value_counts_top.append(pd.DataFrame(columns=value_count_columns,data=[[column,'nulls',null_count]]))
        ## add % counts

        if(total_count!=0):
                value_counts_top['% counts'] = value_counts_top.value_count/float(total_count) * 100
        else:
                value_counts_top['% counts'] = 0

            ## merge temp_dataframe and value_counts_top, then insert into discrete_dataframe
        temp_discrete_dataFrame = temp_dataFrame.merge(how='left',left_on='Column Name',right_on='Column Name',right=value_counts_top)

        discrete_dataFrame = discrete_dataFrame.append(temp_discrete_dataFrame)
df=data_file.describe(percentiles=[.1,.2,.25,.3,.4,.6,.7,.75,.8,.9]).T.reset_index()
df=df.drop(labels="count",axis=1)
df2=pd.merge(discrete_dataFrame,df,how='left',left_on='Column Name',right_on='index')
Q1 = data_file.quantile(0.25)
Q3 = data_file.quantile(0.75)
IQR = Q3 - Q1
outliers=pd.DataFrame(data=((data_file< (Q1 - 1.5 * IQR)) | (data_file > (Q3 + 1.5 * IQR))).sum(),columns=["Outliers_count"]).reset_index()
df3=pd.merge(df2,outliers,how='left',left_on='Column Name',right_on='index')
df3=df3.drop(labels=['index_x','index_y'],axis=1)
df3['Column_Type']=['Continous']*len(df3)
df3=df3[['Column_Type','Column Name', 'total_count', 'unique_count', 'null_count', '% null',
       'levels', 'value_count', '% counts', 'mean', 'std', 'min',  '50%', 'max',
       'Outliers_count']]
df3=df3.append(discrete_dataFrame_cat)


ValueError: Length mismatch: Expected axis has 2 elements, new values have 3 elements

In [24]:
column_data

Dask Series Structure:
npartitions=10
0      float64
12         ...
        ...   
108        ...
117        ...
Name: closed_at, dtype: float64
Dask Name: getitem, 30 tasks

In [66]:
#Univariate for github_main - Solana
sol_main=df3
sol_main.insert(0, 'Crypto', 'Solana')
sol_main

,Crypto,Column_Type,Column Name,total_count,unique_count,null_count,% null,levels,value_count,% counts,mean,std,min,50%,max,Outliers_count
0,Solana,Continous,id,12,1,0,0.0,121470383,12,100.0,1.214704e+08,0.000000,121470383.0,121470383.0,121470383.0,0.0
1,Solana,Continous,id,12,1,0,0.0,others,0,0.0,1.214704e+08,0.000000,121470383.0,121470383.0,121470383.0,0.0
2,Solana,Continous,id,12,1,0,0.0,nulls,0,0.0,1.214704e+08,0.000000,121470383.0,121470383.0,121470383.0,0.0
3,Solana,Continous,size,12,10,0,0.0,439138,2,16.666667,4.401268e+05,745.634461,439138.0,440593.0,440970.0,0.0
4,Solana,Continous,size,12,10,0,0.0,440738,2,16.666667,4.401268e+05,745.634461,439138.0,440593.0,440970.0,0.0
5,Solana,Continous,size,12,10,0,0.0,439684,1,8.333333,4.401268e+05,745.634461,439138.0,440593.0,440970.0,0.0
6,Solana,Continous,size,12,10,0,0.0,439241,1,8.333333,4.401268e+05,745.634461,439138.0,440593.0,440970.0,0.0
7,Solana,Continous,size,12,10,0,0.0,439317,1,8.333333,4.401268e+05,745.634461,439138.0,440593.0,440970.0,0.0
8,Solana,Continous,size,12,10,0,0.0,440589,1,8.333333,4.401268e+05,745.634461,439138.0,440593.0,440970.0,0.0
9,Solana,Continous,size,12,10,0,0.0,440597,1,8.333333,4.401268e+05,745.634461,439138.0,440593.0,440970.0,0.0


In [71]:
#Univariate for github_main - BTC
btc_main=df3
btc_main.insert(0, 'Crypto', 'BTC')
btc_main

,Crypto,Column_Type,Column Name,total_count,unique_count,null_count,% null,levels,value_count,% counts,mean,std,min,50%,max,Outliers_count
0,BTC,Continous,id,14,1,0,0.0,1181927,14,100.0,1.181927e+06,0.000000,1181927.0,1181927.0,1181927.0,0.0
1,BTC,Continous,id,14,1,0,0.0,others,0,0.0,1.181927e+06,0.000000,1181927.0,1181927.0,1181927.0,0.0
2,BTC,Continous,id,14,1,0,0.0,nulls,0,0.0,1.181927e+06,0.000000,1181927.0,1181927.0,1181927.0,0.0
3,BTC,Continous,size,14,7,0,0.0,228758,4,28.571429,2.288869e+05,105.266303,228758.0,228959.0,228984.0,0.0
4,BTC,Continous,size,14,7,0,0.0,228984,3,21.428571,2.288869e+05,105.266303,228758.0,228959.0,228984.0,0.0
5,BTC,Continous,size,14,7,0,0.0,228959,2,14.285714,2.288869e+05,105.266303,228758.0,228959.0,228984.0,0.0
6,BTC,Continous,size,14,7,0,0.0,228971,2,14.285714,2.288869e+05,105.266303,228758.0,228959.0,228984.0,0.0
7,BTC,Continous,size,14,7,0,0.0,228782,1,7.142857,2.288869e+05,105.266303,228758.0,228959.0,228984.0,0.0
8,BTC,Continous,size,14,7,0,0.0,228813,1,7.142857,2.288869e+05,105.266303,228758.0,228959.0,228984.0,0.0
9,BTC,Continous,size,14,7,0,0.0,228978,1,7.142857,2.288869e+05,105.266303,228758.0,228959.0,228984.0,0.0


In [76]:
#Univariate for github_commits - BTC
btc_commits=df3
btc_commits.insert(0, 'Crypto', 'BTC')
btc_commits

,Crypto,Column_Type,Column Name,total_count,unique_count,null_count,% null,levels,value_count,% counts,mean,std,min,50%,max,Outliers_count
0,BTC,Continous,total,312,42,0,0.0,44,24,7.692308,3.856731e+01,1.546644e+01,4.000000e+00,3.900000e+01,7.200000e+01,0.0
1,BTC,Continous,total,312,42,0,0.0,40,18,5.769231,3.856731e+01,1.546644e+01,4.000000e+00,3.900000e+01,7.200000e+01,0.0
2,BTC,Continous,total,312,42,0,0.0,35,18,5.769231,3.856731e+01,1.546644e+01,4.000000e+00,3.900000e+01,7.200000e+01,0.0
3,BTC,Continous,total,312,42,0,0.0,36,13,4.166667,3.856731e+01,1.546644e+01,4.000000e+00,3.900000e+01,7.200000e+01,0.0
4,BTC,Continous,total,312,42,0,0.0,20,12,3.846154,3.856731e+01,1.546644e+01,4.000000e+00,3.900000e+01,7.200000e+01,0.0
5,BTC,Continous,total,312,42,0,0.0,32,12,3.846154,3.856731e+01,1.546644e+01,4.000000e+00,3.900000e+01,7.200000e+01,0.0
6,BTC,Continous,total,312,42,0,0.0,22,12,3.846154,3.856731e+01,1.546644e+01,4.000000e+00,3.900000e+01,7.200000e+01,0.0
7,BTC,Continous,total,312,42,0,0.0,60,12,3.846154,3.856731e+01,1.546644e+01,4.000000e+00,3.900000e+01,7.200000e+01,0.0
8,BTC,Continous,total,312,42,0,0.0,39,12,3.846154,3.856731e+01,1.546644e+01,4.000000e+00,3.900000e+01,7.200000e+01,0.0
9,BTC,Continous,total,312,42,0,0.0,42,11,3.525641,3.856731e+01,1.546644e+01,4.000000e+00,3.900000e+01,7.200000e+01,0.0


In [80]:
#Univariate for github_commits - Solana
sol_commits=df3
sol_commits.insert(0, 'Crypto', 'Solana')
sol_commits

,Crypto,Column_Type,Column Name,total_count,unique_count,null_count,% null,levels,value_count,% counts,mean,std,min,50%,max,Outliers_count
0,Solana,Continous,total,416,45,0,0.0,82,24,5.769231,7.089663e+01,2.461357e+01,1.000000e+01,7.150000e+01,1.310000e+02,17.0
1,Solana,Continous,total,416,45,0,0.0,65,17,4.086538,7.089663e+01,2.461357e+01,1.000000e+01,7.150000e+01,1.310000e+02,17.0
2,Solana,Continous,total,416,45,0,0.0,99,16,3.846154,7.089663e+01,2.461357e+01,1.000000e+01,7.150000e+01,1.310000e+02,17.0
3,Solana,Continous,total,416,45,0,0.0,53,16,3.846154,7.089663e+01,2.461357e+01,1.000000e+01,7.150000e+01,1.310000e+02,17.0
4,Solana,Continous,total,416,45,0,0.0,76,16,3.846154,7.089663e+01,2.461357e+01,1.000000e+01,7.150000e+01,1.310000e+02,17.0
5,Solana,Continous,total,416,45,0,0.0,95,16,3.846154,7.089663e+01,2.461357e+01,1.000000e+01,7.150000e+01,1.310000e+02,17.0
6,Solana,Continous,total,416,45,0,0.0,83,16,3.846154,7.089663e+01,2.461357e+01,1.000000e+01,7.150000e+01,1.310000e+02,17.0
7,Solana,Continous,total,416,45,0,0.0,60,16,3.846154,7.089663e+01,2.461357e+01,1.000000e+01,7.150000e+01,1.310000e+02,17.0
8,Solana,Continous,total,416,45,0,0.0,69,16,3.846154,7.089663e+01,2.461357e+01,1.000000e+01,7.150000e+01,1.310000e+02,17.0
9,Solana,Continous,total,416,45,0,0.0,70,16,3.846154,7.089663e+01,2.461357e+01,1.000000e+01,7.150000e+01,1.310000e+02,17.0


In [86]:
#Univariate for github_issues - Solana
sol_issues=df3
sol_issues.insert(0, 'Crypto', 'Solana')
sol_issues

,Crypto,Column_Type,Column Name,total_count,unique_count,null_count,% null,levels,value_count,% counts,mean,std,min,50%,max,Outliers_count
0,Solana,Continous,id,87,87,0,0.0,1661541619,1,1.149425,1.663361e+09,3.539410e+06,1.655990e+09,1.663322e+09,1.668952e+09,0.0
1,Solana,Continous,id,87,87,0,0.0,1658452161,1,1.149425,1.663361e+09,3.539410e+06,1.655990e+09,1.663322e+09,1.668952e+09,0.0
2,Solana,Continous,id,87,87,0,0.0,1661296052,1,1.149425,1.663361e+09,3.539410e+06,1.655990e+09,1.663322e+09,1.668952e+09,0.0
3,Solana,Continous,id,87,87,0,0.0,1661293686,1,1.149425,1.663361e+09,3.539410e+06,1.655990e+09,1.663322e+09,1.668952e+09,0.0
4,Solana,Continous,id,87,87,0,0.0,1661257166,1,1.149425,1.663361e+09,3.539410e+06,1.655990e+09,1.663322e+09,1.668952e+09,0.0
5,Solana,Continous,id,87,87,0,0.0,1661142003,1,1.149425,1.663361e+09,3.539410e+06,1.655990e+09,1.663322e+09,1.668952e+09,0.0
6,Solana,Continous,id,87,87,0,0.0,1661120195,1,1.149425,1.663361e+09,3.539410e+06,1.655990e+09,1.663322e+09,1.668952e+09,0.0
7,Solana,Continous,id,87,87,0,0.0,1661106152,1,1.149425,1.663361e+09,3.539410e+06,1.655990e+09,1.663322e+09,1.668952e+09,0.0
8,Solana,Continous,id,87,87,0,0.0,1661093712,1,1.149425,1.663361e+09,3.539410e+06,1.655990e+09,1.663322e+09,1.668952e+09,0.0
9,Solana,Continous,id,87,87,0,0.0,1661014315,1,1.149425,1.663361e+09,3.539410e+06,1.655990e+09,1.663322e+09,1.668952e+09,0.0


In [83]:
#Univariate for github_issues - Solana
btc_issues=df3
btc_issues.insert(0, 'Crypto', 'BTC')
btc_issues

,Crypto,Column_Type,Column Name,total_count,unique_count,null_count,% null,levels,value_count,% counts,mean,std,min,50%,max,Outliers_count
0,BTC,Continous,id,45,45,0,0.0,1661601514,1,2.222222,1.657467e+09,7.386097e+06,1.644651e+09,1.656346e+09,1.669057e+09,0.0
1,BTC,Continous,id,45,45,0,0.0,1661656788,1,2.222222,1.657467e+09,7.386097e+06,1.644651e+09,1.656346e+09,1.669057e+09,0.0
2,BTC,Continous,id,45,45,0,0.0,1661381553,1,2.222222,1.657467e+09,7.386097e+06,1.644651e+09,1.656346e+09,1.669057e+09,0.0
3,BTC,Continous,id,45,45,0,0.0,1658927974,1,2.222222,1.657467e+09,7.386097e+06,1.644651e+09,1.656346e+09,1.669057e+09,0.0
4,BTC,Continous,id,45,45,0,0.0,1658002064,1,2.222222,1.657467e+09,7.386097e+06,1.644651e+09,1.656346e+09,1.669057e+09,0.0
5,BTC,Continous,id,45,45,0,0.0,1657857428,1,2.222222,1.657467e+09,7.386097e+06,1.644651e+09,1.656346e+09,1.669057e+09,0.0
6,BTC,Continous,id,45,45,0,0.0,1657835564,1,2.222222,1.657467e+09,7.386097e+06,1.644651e+09,1.656346e+09,1.669057e+09,0.0
7,BTC,Continous,id,45,45,0,0.0,1656345711,1,2.222222,1.657467e+09,7.386097e+06,1.644651e+09,1.656346e+09,1.669057e+09,0.0
8,BTC,Continous,id,45,45,0,0.0,1656220659,1,2.222222,1.657467e+09,7.386097e+06,1.644651e+09,1.656346e+09,1.669057e+09,0.0
9,BTC,Continous,id,45,45,0,0.0,1656067373,1,2.222222,1.657467e+09,7.386097e+06,1.644651e+09,1.656346e+09,1.669057e+09,0.0
